In [1]:
!pip3 install pysubs2

In [34]:
import pysubs2

In [84]:
subs = pysubs2.load("Primer_Sub.srt", encoding="utf-8")
subs.shift(s=-16)

In [85]:
## The problem is that the last subtitles should be at 01:12:16 and not at 1:15:15
## so we can think to decode in normal number, compress and then reencode.


In [86]:
for sub in subs:
    print(sub)

<SSAEvent type=Dialogue start=0:00:18 end=0:00:20 text='(PHONE RINGING)'>
<SSAEvent type=Dialogue start=0:00:26 end=0:00:28 text="{\\i1}(MAN) Here's what's going to happen.{\\i0}">
<SSAEvent type=Dialogue start=0:00:28 end=0:00:31 text="{\\i1}I'm gonna read this,\\Nand you're gonna listen,{\\i0}">
<SSAEvent type=Dialogue start=0:00:31 end=0:00:33 text="{\\i1}and you're gonna stay on the line,\\Nand you're not gonna interrupt.{\\i0}">
<SSAEvent type=Dialogue start=0:00:34 end=0:00:36 text="{\\i1}You're not gonna speak for any reason.{\\i0}">
<SSAEvent type=Dialogue start=0:00:36 end=0:00:37 text='{\\i1}Some of this you know.{\\i0}'>
<SSAEvent type=Dialogue start=0:00:38 end=0:00:40 text="{\\i1}I'm gonna start\\Nat the top of the page.{\\i0}">
<SSAEvent type=Dialogue start=0:00:43 end=0:00:45 text='{\\i1}Meticulous, yes.{\\i0}'>
<SSAEvent type=Dialogue start=0:00:45 end=0:00:48 text='{\\i1}Methodical. Educated.\\NThey were these things.{\\i0}'>
<SSAEvent type=Dialogue start=0:00:49 end=0

In [87]:
subs

<SSAFile with 1218 events and 1 styles, last timestamp 1:15:22>

In [88]:
#subs.shift(s=-20)

In [89]:
subs.save("Primer_shifted.srt")

In [90]:
sub

<SSAEvent type=Dialogue start=1:15:19 end=1:15:22 text='{\\i1}And if you look, you will not find me.{\\i0}'>

In [91]:
def decode_to_s(ts): ## ts stands for timestamp
    ts_split= ts.split(sep=":")
    ts_second = int(ts_split[2])+(int(ts_split[1])*60)+(int(ts_split[0])*3600)
    return ts_second

In [92]:
def encode_to_ts(t_second):
    hour = int(t_second/3600)
    reminder_hour = t_second%3600
    minute = int(reminder_hour/60)
    second = (reminder_hour%60)
    if minute < 10:
        minute = f'0{minute}'
    if second < 10:
        second = f'0{second}'
    timestamp = f'{hour}:{minute}:{second}'
    return timestamp

In [93]:
## Now the we wrote the enconding/decoding function we can easly find the compression factor:

In [94]:
TS_FILE = decode_to_s('01:15:22')
TS_MOVIE = decode_to_s('01:12:16') ## This value was taken empyrically wacthing the movie xD

CP = TS_MOVIE/TS_FILE ## Now this value can be multiply by the value of the timestamp in the file to make a new one
                       ### with the correct values.

In [95]:
sub

<SSAEvent type=Dialogue start=1:15:19 end=1:15:22 text='{\\i1}And if you look, you will not find me.{\\i0}'>

In [96]:
def modify_single_sub(single_sub, CP):
    sub_splitted = str(single_sub).split(sep=" ")
    text_splitted = str(single_sub).split(sep="text=")
    start_ts_list = sub_splitted[2].split(sep="=")
    start_ts = start_ts_list[1]
    
    end_ts_list = sub_splitted[3].split(sep="=")
    end_ts = end_ts_list[1]
    
    start_second = decode_to_s(start_ts)
    end_second = decode_to_s(end_ts)
    
    new_start_second = int(start_second*CP)
    new_end_second = int(end_second*CP)
    
    new_start_ts = encode_to_ts(int(start_second*CP))
    new_end_ts = encode_to_ts(int(end_second*CP))
    
    #print(sub_splitted)
    
    sub_splitted[2] = f'start={new_start_ts}'
    sub_splitted[3] = f'end={new_end_ts}'
    #print(sub_splitted)
    
    text = text_splitted[1]
    new_sub_line = ' '.join(sub_splitted)
    #print(text_splitted[1])
    #print(new_sub_line)
    #print(start_ts, end_ts, start_second, end_second, new_start_ts, new_end_ts)
    return new_sub_line, new_start_second, new_end_second, text

In [97]:
modify_single_sub(sub, CP)

("<SSAEvent type=Dialogue start=1:12:13 end=1:12:16 text='{\\\\i1}And if you look, you will not find me.{\\\\i0}'>",
 4333,
 4336,
 "'{\\\\i1}And if you look, you will not find me.{\\\\i0}'>")

In [98]:
new_sub = []
SSAFile = pysubs2.SSAFile()
for n,single_sub in enumerate(subs):
    #print(single_sub)
    new_single_sub, start_new, end_new, text_ = modify_single_sub(single_sub, CP)
    new_sub.append(new_single_sub)
    SSAEvent = pysubs2.SSAEvent(start=int(start_new*1000), end=int(end_new*1000), text=text_)
    SSAFile.insert(index=n+1, value=SSAEvent)

In [99]:
SSAFile.save("PRIME_by_SignorG.srt")